# Where is the ISS currently?

In this mini python project I will show you how to use the `ISS API` to do the following:
- Find the current people on board the ISS
- Determine the location of the ISS

We will be using the following endpoints:
- http://api.open-notify.org/iss-now.json
    - Current location of the ISS
- http://api.open-notify.org/astros.json
    - Current people on board the ISS

In [2]:
try:
    import requests as rq
    import json
    from datetime import datetime as dt
    import pandas as pd
    from urllib import parse
    ## print('[SUCCESSFULLY IMPORTED]')
    ## return nothing for a succesful import
except ImportError as ie:
    print(f'Import Library Error: {ie}')
    
    #pip3 install --upgrade requests


In [3]:
class ISS:
    def __init__(self, state = ''): 
        # ------- SET ATTRIBUTES -------
        #-------------------------------
        self.url_current_loc = 'http://api.open-notify.org/iss-now.json'
        self.url_current_astros = 'http://api.open-notify.org/astros.json'
        self.url_locater = 'https://nominatim.openstreetmap.org/reverse'

     
    def check_url_stat(self, url, params = ''):
        url_stat = rq.get(url + params)
        try:
            if url_stat.status_code == 200:
                json_info = url_stat.json()
                return json_info
        except Exception as e:
            raise e
    
    def json_format(self, json_info):
        txt = json.dumps(json_info, sort_keys = True, indent = 4)
        return txt
    
    def create_df_iss_astro_info(self) -> pd.DataFrame:
        iss_info = self.check_url_stat(self.url_current_astros)
        if iss_info:
            people = iss_info['people']
            
            df = pd.DataFrame.from_dict(people, orient = 'columns')
            return df
        
    def current_people(self):
        df = self.create_df_iss_astro_info()
        df_people = df.copy()
        df_people['Current Astronauts in Space'] = df.name
        
        df_people.drop(columns = ['name', 'craft'], inplace = True)
        return df_people
    
    def current_crafts(self):
        df = self.create_df_iss_astro_info()
        df_crafts = df.copy()
        df_crafts['CRAFTS'] = df.craft
        
        return df_crafts.groupby('CRAFTS')['name'].count()
    

    def create_df_iss_loc_info(self):
        iss_loc_info = self.check_url_stat(self.url_current_loc)
        if iss_loc_info:
            location = iss_loc_info['iss_position']
            time = iss_loc_info['timestamp']
            time_stamp = dt.fromtimestamp(time)
            
            
            df = pd.DataFrame.from_dict([location])
            df['timestamp'] = time_stamp
            return df
        
    def get_current_iss_loc(self):
        df = self.create_df_iss_loc_info()
        lat = df['latitude'][0]
        lon = df['longitude'][0]
        params = {'format': 'json', 
                 'lat': lat,
                 'lon': lon,
                 'zoom': 18,
                 'addressdetails': 1}
        new_params = f'?{parse.urlencode(params)}'
        info = self.check_url_stat(self.url_locater, new_params)
        return info
#         return f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1'
#         return lat, lon
#         return f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}'
#         return f'https://forecast.weather.gov/MapClick.php?lat={lat}&lon={lon}'

    def get_iss_loc(self):
        info = self.get_current_iss_loc()
        df = info
        if 'address' not in df:
            return 'CURRENTLY HOVERING OVER UNKNOWN TERRITORY ACCORDING TO API'
        else:
            df = info['address']
            if 'country' not in df:
                return df['locatlity']
            return df['country']
        

In [4]:
# Testing each function individually
iss = ISS()

print(iss.json_format(iss.check_url_stat('http://api.open-notify.org/astros.json')))
print(iss.current_people())
print()
print(iss.current_crafts())
print()
print(iss.create_df_iss_astro_info())
print()
print(iss.create_df_iss_loc_info())
print()
print(iss.get_current_iss_loc())
print()
print(iss.get_iss_loc())

{
    "message": "success",
    "number": 10,
    "people": [
        {
            "craft": "ISS",
            "name": "Oleg Artemyev"
        },
        {
            "craft": "ISS",
            "name": "Denis Matveev"
        },
        {
            "craft": "ISS",
            "name": "Sergey Korsakov"
        },
        {
            "craft": "ISS",
            "name": "Kjell Lindgren"
        },
        {
            "craft": "ISS",
            "name": "Bob Hines"
        },
        {
            "craft": "ISS",
            "name": "Samantha Cristoforetti"
        },
        {
            "craft": "ISS",
            "name": "Jessica Watkins"
        },
        {
            "craft": "Tiangong",
            "name": "Cai Xuzhe"
        },
        {
            "craft": "Tiangong",
            "name": "Chen Dong"
        },
        {
            "craft": "Tiangong",
            "name": "Liu Yang"
        }
    ]
}
  Current Astronauts in Space
0               Oleg Artemyev
1        

- https://stackoverflow.com/questions/3513134/how-to-reverse-geocode-without-google
- https://developers.google.com/maps/documentation/geocoding/requests-reverse-geocoding
- https://nominatim.openstreetmap.org/ui/search.html
- https://stackoverflow.com/questions/2506379/add-params-to-given-url-in-python
- https://nominatim.openstreetmap.org/ui/reverse.html?zoom=18


In [5]:
def main():
    iss = ISS()
    print(iss.current_people())
    print()
    print(iss.current_crafts())
    print()
    print(iss.get_iss_loc())

In [6]:
if __name__ == '__main__':
    main()

  Current Astronauts in Space
0               Oleg Artemyev
1               Denis Matveev
2             Sergey Korsakov
3              Kjell Lindgren
4                   Bob Hines
5      Samantha Cristoforetti
6             Jessica Watkins
7                   Cai Xuzhe
8                   Chen Dong
9                    Liu Yang

CRAFTS
ISS         7
Tiangong    3
Name: name, dtype: int64

CURRENTLY HOVERING OVER UNKNOWN TERRITORY ACCORDING TO API
